Web Scraping

Scraping job postings from one of Mongolia's top job boards: Zangia.mn (formerly BizNetwork). I will collect the following features from the site:

- Job title
- Job description
- Job sector
- Salary range

If you go to https://www.zangia.mn/job/list you will see all the job listings. There are several pages of listings. 

here is my process

1. Made a list of job post results.
2. Scraped the listing URLs from the job list pages (1 through n).
3. Using the resulting URL list to scrape the features to a dataframe.



### Libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen

#### main

In [2]:
def collect(a):
    source = '"""'+str(a)+'"""'
    soup = BeautifulSoup(source)
    return soup.find("span").text

In [3]:
features =  ['Job title', 'Job description', 'Job sector', 'Salary range']
collected_inf  = pd.DataFrame(columns = features)

In [4]:
collected_inf

,Job title,Job description,Job sector,Salary range


In [5]:
for i in range(1,86):
    list_page = requests.get(f'https://www.zangia.mn/job/list/pg.{str(i)}').content.decode("utf-8")
    sp = BeautifulSoup(list_page, "html.parser")  
    ad = sp.find_all("div", attrs={"class": "ad"})
    for element in ad:
        source = '"""'+str(element)+'"""'
        source_soup = BeautifulSoup(source)
        link = source_soup.find('a', href=True) 
        each_pages = requests.get('https://www.zangia.mn/'+str(link['href'])).content.decode("utf-8")
        job = BeautifulSoup(each_pages, "html.parser")
        
        
        inf = job.find("div", attrs={"class": "details"})
        str_inf = '"""'+str(inf)[21:]+'"""'
        inf_soup = BeautifulSoup(str_inf)        
        all_inf = inf_soup.find_all("div")
        for inf in all_inf:
            if "Салбар" in str(inf):
                job_sector = collect(inf)
            if "Цалин" in str(inf):
                salary_range = collect(inf)
        job_title = job.find('h3').text
        job_description = job.find("div", attrs={"class": "section"}).text
        
        def build_df(collected_inf, row):
            insert_loc = collected_inf.index.max()
            if pd.isna(insert_loc):
                collected_inf.loc[0] = row
            else:
                collected_inf.loc[insert_loc + 1] = row
                
        build_df(collected_inf,[job_title,job_description,job_sector,salary_range])  
        
        collected_inf

In [6]:
collected_inf

,Job title,Job description,Job sector,Salary range
0,Веб-д суурилсан програм хөгжүүлэгч (Cash One app),Гүйцэтгэх үндсэн үүрэгВеб-д суурилсан програм ...,"Мэдээлэл технологи, Харилцаа холбоо","1,500,000 - 1,800,000"
1,Mobile-app хөгжүүлэгч,Гүйцэтгэх үндсэн үүрэгCash One зээлийн applica...,"Мэдээлэл технологи, Харилцаа холбоо","1,500,000 - 1,800,000"
2,Програм хөгжүүлэгч,"Гүйцэтгэх үндсэн үүрэгСистемийн шинжилгээ, зох...","Мэдээлэл технологи, Харилцаа холбоо","1,000,000 - 1,200,000"
3,Мобайл аппликейшн хөгжүүлэгч,Гүйцэтгэх үндсэн үүрэг\nМобайл аппликейшн хийх...,"Мэдээлэл технологи, Харилцаа холбоо","4,000,000-аас дээшТохиролцоно"
4,"ДОТООД ХЯНАЛТ, ЧАНАРЫН МЕНЕЖЕР АЖИЛД АВНА",Гүйцэтгэх үндсэн үүрэгЛабораторийн үйл ажиллаг...,"Үйлдвэрлэл, Дэд бүтэц","1,200,000 - 1,500,000"
...,...,...,...,...
4252,Борлуулалтын менежер,Гүйцэтгэх үндсэн үүрэгБорлуулалтыг менежментий...,"Худалдаа, Борлуулалт","1,200,000 - 1,500,000"
4253,МЭДЭЭЛЭЛ ТЕХНОЛОГИЙН ИНЖЕНЕР,Гүйцэтгэх үндсэн үүрэгАжлын байрны тодорхойлол...,"Мэдээлэл технологи, Харилцаа холбоо","1,200,000 - 1,500,000"
4254,"Сантехникч, техникийн ажилтан",Гүйцэтгэх үндсэн үүрэгКэмпийн сантехникийн асу...,"Барилга, Үл хөдлөх хөрөнгө","800,000 - 1,000,000"
4255,Хөгжмийн багш,"Гүйцэтгэх үндсэн үүрэг-Төгөлдөр хуур, гитар зэ...","Боловсрол, Шинжлэх ухаан","600,000 - 800,000Тохиролцоно"
